In [ ]:
from flask import Flask, render_template, redirect, request, url_for, send_file
from flask import jsonify, json
from werkzeug.utils import secure_filename
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import time

UPLOAD_FOLDER = 'Uploaded_Files'
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Load the TensorFlow model
model = load_model('model/df_model.h5')

def detectFakeVideo(videoPath):
    # Video processing and prediction logic
    cap = cv2.VideoCapture(videoPath)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    # Process frames and make predictions
    predictions = []
    for frame in frames:
        img = cv2.resize(frame, (112, 112))
        img = np.expand_dims(img, axis=0)
        pred = model.predict(img)
        predictions.append(pred)
    return predictions

@app.route('/', methods=['POST', 'GET'])
def homepage():
    if request.method == 'GET':
        return render_template('index.html')
    return render_template('index.html')

@app.route('/Detect', methods=['POST', 'GET'])
def DetectPage():
    if request.method == 'POST':
        video = request.files['video']
        video_filename = secure_filename(video.filename)
        video.save(os.path.join(app.config['UPLOAD_FOLDER'], video_filename))
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], video_filename)
        predictions = detectFakeVideo(video_path)
        # Process predictions and return results
        return render_template('index.html', data=predictions)
    return render_template('index.html')

app.run(port=3000)
